# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [276]:
import pandas as pd
import numpy as np
from datetime import datetime


To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [277]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

## How many rows are there in the GBvideos.csv after removing duplications?

load data GBvideos.csv

In [278]:
vdo_df = pd.read_csv('data/GBvideos.csv')

check rows and columns info and shap

In [279]:
vdo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38916 entries, 0 to 38915
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   video_id                38916 non-null  object
 1   trending_date           38916 non-null  object
 2   title                   38916 non-null  object
 3   channel_title           38916 non-null  object
 4   category_id             38916 non-null  int64 
 5   publish_time            38916 non-null  object
 6   tags                    38916 non-null  object
 7   views                   38916 non-null  int64 
 8   likes                   38916 non-null  int64 
 9   dislikes                38916 non-null  int64 
 10  comment_count           38916 non-null  int64 
 11  thumbnail_link          38916 non-null  object
 12  comments_disabled       38916 non-null  bool  
 13  ratings_disabled        38916 non-null  bool  
 14  video_error_or_removed  38916 non-null  bool  
 15  de

In [280]:
vdo_df.shape

(38916, 16)

Drop duplicated rows and print out the unique rows

In [281]:
vdo_df_unique = vdo_df.drop_duplicates()
vdo_df_unique.shape[0]

38745

## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

check the head of rows which have dislikes more than likes

In [282]:
vdo_df_unique[vdo_df_unique['likes'] < vdo_df_unique['dislikes']][['title','likes', 'dislikes']].head()

,title,likes,dislikes
91,Bone on Labour HQ,1160,1164
110,After Show: Did Kate Hudson Date Brad Pitt? | ...,184,187
161,Svjetska ekskluziva: Provala u stan Dejana Lov...,83,108
309,Bone on Labour HQ,1168,1170
327,After Show: Did Kate Hudson Date Brad Pitt? | ...,186,190


use aggergate function "nunique" to find the unique title which has dislikes more than likes

In [283]:
mapper = {
    'title': 'nunique'
}
title_dislikes_more_likes = vdo_df_unique[vdo_df_unique['likes'] < vdo_df_unique['dislikes']].agg(mapper)
display(int(title_dislikes_more_likes))

/var/folders/s4/220_46zj42nfr21hd0l1_ks40000gn/T/ipykernel_45214/3906632881.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  display(int(title_dislikes_more_likes))


56

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

use condition "where trending_data == '18.22.01' and comment_count > 10000" to find rows and check this head to verify

In [284]:
vdo_selected_df = vdo_df_unique[(vdo_df_unique['trending_date'] == '18.22.01') & (vdo_df_unique['comment_count'] > 10_000)][['title', 'trending_date', 'comment_count']]
vdo_selected_df.head()

,title,trending_date,comment_count
13392,My Horibal Speling,18.22.01,70565
13394,The Smallest Bird you have ever seen,18.22.01,10121
13397,ELDERS REACT TO UGANDAN KNUCKLES MEMES,18.22.01,14099
13398,Primitive Technology: A-frame hut,18.22.01,13839
13411,"J. Balvin, Jeon, Anitta - Machika",18.22.01,69932


print out the rows which are true from condition

In [285]:
vdo_selected_df.shape[0]

28

## Which date that has the minimum average number of comments per VDO?

group by trending_date then calculate the average of comment_count per day

In [286]:
comment_avg_per_dt = vdo_df_unique.groupby('trending_date')[['trending_date', 'comment_count']].mean('comment_count')
comment_avg_per_dt

,comment_count
trending_date,
17.01.12,9528.712121
17.02.12,9077.180905
17.03.12,9132.688442
17.04.12,9276.555000
17.05.12,9510.740000
...,...
18.30.04,18460.151515
18.30.05,23361.103659
18.31.01,9318.939698


find the minimum average day by using idxmin() then select only value at np.array position 0

In [287]:
orig_min_avg_comment_dt = (comment_avg_per_dt.idxmin()).values[0]
min_avg_comment_dt =  datetime.strptime(orig_min_avg_comment_dt, '%y.%d.%m')
formatted_min_avg_comment_dt = min_avg_comment_dt.strftime('%d-%m-%Y')
print(f'the date that has the minimum average number of comments per VDO is {orig_min_avg_comment_dt } or {formatted_min_avg_comment_dt}')

the date that has the minimum average number of comments per VDO is 17.15.11 or 15-11-2017


## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

load data/GB_category_id.json into cat_df

In [288]:
import json 

with open('data/GB_category_id.json') as fd:
    cat = json.load(fd)

cat_list = []
for d in cat['items']:
    cat_list.append((int(d['id']), d['snippet']['title']))

cat_df= pd.DataFrame(cat_list, columns=['id', 'category'])

inner join vdo_df_unique and cat_df by using key category_id and id respectively

In [289]:
vdo_df_unique_with_cat = vdo_df_unique.merge(cat_df, left_on='category_id', right_on='id')
vdo_df_unique_with_cat.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,id,category
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...,26,Howto & Style
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...,24,Entertainment
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...,10,Music
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...,17,Sports
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...,25,News & Politics


filter category only Sports and Comedy

In [290]:
sports_comedy_df = vdo_df_unique_with_cat[vdo_df_unique_with_cat['category'].isin(['Sports','Comedy'])]
sports_comedy_df['category'].head()

3     Sports
10    Sports
18    Comedy
26    Sports
28    Comedy
Name: category, dtype: object

group by trending_date and category respectively and select columns views then use unpack() to pivot a level of the index labels

In [291]:
daily_views = sports_comedy_df.groupby(['trending_date', 'category'])['views'].sum().unstack(level=-1)
daily_views

category,Comedy,Sports
trending_date,,
17.01.12,10889970,9879808
17.02.12,13302391,10856461
17.03.12,14259944,10875249
17.04.12,16617431,11399173
17.05.12,19089597,11261410
...,...,...
18.30.04,17802688,15114095
18.30.05,16427316,5153521
18.31.01,14840766,8431519


sum days that there are more total daily views of Sports  than Comady

In [292]:
daily_views_sports_more_comedy = (daily_views['Sports'] > daily_views['Comedy']).sum()
display(int(daily_views_sports_more_comedy))

49